In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
from scipy.stats import t
from scipy.stats import f

In [2]:
df = pd.read_csv("https://online.stat.psu.edu/onlinecourses/sites/stat501/files/data/coolhearts.txt",sep='\t')
df.head()

,Inf,Area,Group,X2,X3
0,0.119,0.34,3,0,0
1,0.190,0.64,3,0,0
2,0.395,0.76,3,0,0
3,0.469,0.83,3,0,0
4,0.130,0.73,3,0,0


In [3]:
df.columns=['y','X1','Group','X2','X3']
df.head()

,y,X1,Group,X2,X3
0,0.119,0.34,3,0,0
1,0.190,0.64,3,0,0
2,0.395,0.76,3,0,0
3,0.469,0.83,3,0,0
4,0.130,0.73,3,0,0


In [4]:
full_model = 'y~X1+X2+X3'

In [5]:
AdjANOVA = sm.stats.anova_lm(ols(full_model, data=df).fit(), typ=2) #partial SS
SeqANOVA = sm.stats.anova_lm(ols(full_model, data=df).fit(), typ=1) #sequential SS
display(AdjANOVA, SeqANOVA)

,sum_sq,df,F,PR(>F)
X1,0.637420,1.0,32.753601,0.000004
X2,0.297327,1.0,15.278050,0.000536
X3,0.019814,1.0,1.018138,0.321602
Residual,0.544910,28.0,NaN,NaN


,df,sum_sq,mean_sq,F,PR(>F)
X1,1.0,0.624924,0.624924,32.111540,0.000005
X2,1.0,0.314534,0.314534,16.162214,0.000398
X3,1.0,0.019814,0.019814,1.018138,0.321602
Residual,28.0,0.544910,0.019461,NaN,NaN


In [6]:
y = df.y
y_hat = ols(full_model, data=df).fit().fittedvalues
y_bar = df.y.mean()

In [7]:
n = len(y_hat)
p = ols(full_model, data=df).fit().df_model
k = p + 1
df_error = n - k

In [8]:
SSR = np.sum((y_hat - y_bar)**2)
SSR

0.9592721947639449

In [9]:
SSE = np.sum((y_hat - y)**2)
SSE

0.5449095239860544

In [10]:
MSR = SSR / p
MSR

0.3197573982546483

In [11]:
MSE = SSE / df_error
MSE

0.01946105442807337

In [12]:
#SeqSSR(X1)
SeqSSR_X1 = np.sum((ols('y~X1', data=df).fit().fittedvalues - y_bar)**2)
SeqSSR_X1

0.6249244253810227

In [13]:
#SeqSSR(X2|X1)
SeqSSR_X2 = np.sum((ols('y~X1+X2', data=df).fit().fittedvalues - y_bar)**2)-\
np.sum((ols('y~X1', data=df).fit().fittedvalues - y_bar)**2)
SeqSSR_X2

0.3145337309953077

In [14]:
#SeqSS(X3|X1,X2)
SeqSSR_X3 = np.sum((ols('y~X1+X2+X3', data=df).fit().fittedvalues - y_bar)**2)-\
np.sum((ols('y~X1+X2', data=df).fit().fittedvalues - y_bar)**2)
SeqSSR_X3

0.01981403838761453

In [15]:
#AdjSSR(X1|X2,X3)
AdjSSR_X1 = np.sum((ols('y~X1+X2+X3', data=df).fit().fittedvalues - y_bar)**2)-\
np.sum((ols('y~X2+X3', data=df).fit().fittedvalues - y_bar)**2)
AdjSSR_X1

0.6374196032866714

In [16]:
#AdjSSR(X2|X1,X3)
AdjSSR_X2 = np.sum((ols('y~X1+X2+X3', data=df).fit().fittedvalues - y_bar)**2)-\
np.sum((ols('y~X1+X3', data=df).fit().fittedvalues - y_bar)**2)
AdjSSR_X2

0.29732696790825885

In [17]:
#AdjSSR(X3|X1,X2)
AdjSSR_X3 = np.sum((ols('y~X1+X2+X3', data=df).fit().fittedvalues - y_bar)**2)-\
np.sum((ols('y~X1+X2', data=df).fit().fittedvalues - y_bar)**2)
AdjSSR_X3

0.01981403838761453

In [18]:
coefficients = ols('y~X1+X2+X3', data=df).fit().params
coefficients

Intercept   -0.134536
X1           0.612655
X2          -0.243482
X3          -0.065656
dtype: float64

In [19]:
std_errors = ols('y~X1+X2+X3', data=df).fit().bse
std_errors

Intercept    0.104022
X1           0.107050
X2           0.062292
X3           0.065068
dtype: float64

In [20]:
t_stats = coefficients / std_errors
t_stats

Intercept   -1.293351
X1           5.723076
X2          -3.908715
X3          -1.009028
dtype: float64

In [21]:
p_values = ols('y~X1+X2+X3', data=df).fit().pvalues
p_values

Intercept    0.206459
X1           0.000004
X2           0.000536
X3           0.321602
dtype: float64

In [22]:
{param: 2 * (1 - t.cdf(abs(t_stats[param]), n-k)) for param in ['X1','X2','X3']}

{'X1': 3.865441203787512e-06,
 'X2': 0.0005364535806009485,
 'X3': 0.32160181555623235}

In [23]:
t_stats**2

Intercept     1.672756
X1           32.753601
X2           15.278050
X3            1.018138
dtype: float64

In [24]:
AdjF_X1 = AdjSSR_X1 / MSE
AdjF_X2 = AdjSSR_X2 / MSE
AdjF_X3 = AdjSSR_X3 / MSE
AdjF_X1, AdjF_X2, AdjF_X3

(32.75360056375812, 15.278050272515166, 1.0181379668221864)

In [25]:
SeqF_X1 = SeqSSR_X1 / MSE
SeqF_X2 = SeqSSR_X2 / MSE
SeqF_X3 = SeqSSR_X3 / MSE
SeqF_X1, SeqF_X2, SeqF_X3

(32.111539880364525, 16.162214239613853, 1.0181379668221864)

In [26]:
{SeqF: 1 - f.cdf(SeqF, 1, n-k) for SeqF in [SeqF_X1, SeqF_X2, SeqF_X3]}

{32.111539880364525: 4.503703467828579e-06,
 16.162214239613853: 0.000398016680720259,
 1.0181379668221864: 0.321601815556249}

In [27]:
{AdjF: 1 - f.cdf(AdjF, 1, n-k) for AdjF in [AdjF_X1, AdjF_X2, AdjF_X3]}

{32.75360056375812: 3.865441203787512e-06,
 15.278050272515166: 0.0005364535806010595,
 1.0181379668221864: 0.321601815556249}

$$F^*=\left( \dfrac{SSR(F)-SSR(R)}{df_R-df_F}\right)\div\left( \dfrac{SSE(F)}{df_F}\right)$$

$$H_0: \beta_2 = \beta_3 = 0$$

In [28]:
SSRF = np.sum((ols('y~X1+X2+X3', data=df).fit().fittedvalues - y_bar)**2) #full model
SSRR = np.sum((ols('y~X1', data=df).fit().fittedvalues - y_bar)**2)  #reduced model
SSEF = np.sum((ols('y~X1+X2+X3', data=df).fit().fittedvalues - y)**2)
df_full = n - k  #associated with error sum of squares
df_reduced = df_full + 2

Fstar= ((SSRF - SSRR)/(df_reduced - df_full)) / (SSEF/df_full)
p_value = 1 - f.cdf(Fstar, df_reduced - df_full, df_full)
Fstar, p_value

(8.59017610321802, 0.0012328716567309161)